In [1]:
import sys
import os
import csv
import time
import statistics
import numpy as np
from pathlib import Path, PureWindowsPath
from numpy import dot
from numpy.linalg import norm
import networkx as nx
import networkx.algorithms.matching
from collections import Counter
from IPython.display import Image, display
from matplotlib import pyplot as plt
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [3]:
def getArray (path):
    results = []
    with open(path) as csvfile:
        reader = csv.reader(csvfile,csv.QUOTE_NONNUMERIC) # change contents to floats
        for row in reader: # each row is a list
            nums = []
            for i in row: 
                if i :
                    nums.append(int(i))
            results.append(nums)
    return results

In [4]:
def createDirs(fold):
    
    path = "DATASET F1/fold_" +str(fold)
    mal_path = path+ "/20%_test"
    train_path = path+ "/80%_train"
    benign_path = "DATASET F1/specimen_benign/20%_test"
    
    w_mal_path = PureWindowsPath(mal_path)
    w_train_path= PureWindowsPath(train_path)
    w_benign_path = PureWindowsPath(benign_path)
    print( mal_path+"\n"+ train_path+"\n"+ benign_path)
    print( w_mal_path, w_train_path, w_benign_path)

    filename = "default_G_tone_map"
    mapFilename = "default_unique_mapping"
    
    rootDir = [os.path.join(root, name)
             for root, dirs, files in os.walk(w_mal_path)
             for name in files
             if name.endswith(filename+".csv")]
    rootDir.sort()
    # for i in range(len(paths)):
    #     p = [os.path.join(root, name)
    #              for root, dirs, files in os.walk(paths[i]+"/"+path)
    #              for name in files
    #              if name.endswith(filename+".csv")]
    #     p.sort()
    #     rootDirs[i] = p
    # print(rootDirs[10])
    benignDir = [os.path.join(root, name)
                 for root, dirs, files in os.walk(w_benign_path)
                 for name in files
                 if name.endswith(filename+".csv")]
    benignDir.sort()

    # for i in range(len(paths)):
    trainDir = [os.path.join(root, name)
                 for root, dirs, files in os.walk(w_train_path)
                 for name in files
                 if name.endswith(filename+".csv")]
    trainDir.sort()
    #     trainDirs[i] = trainDir
    return [rootDir,benignDir,trainDir]

In [5]:
def createGraphs(dirs, npmode):
    rootDir = dirs[0]
    benignDir = dirs[1]
    trainDir = dirs[2]
    mal_gs = []
    mal_np_gs = []
    ben_gs = []
    train_gs = []
    train_np_gs = []
    ben_np_gs = []
    csm_vals = {}
    for i in range(len(rootDir)):
        A = np.array(getArray(rootDir[i]))
        G = nx.from_numpy_matrix(A)
        mal_gs.append(G)
        mal_np_gs.append(A)
        printProgressBar(i, len(rootDir)-1,prefix = 'malware gs')

    for i in range(len(trainDir)):
        A = np.array(getArray(trainDir[i]))
        G = nx.from_numpy_matrix(A)
        train_gs.append(G)
        train_np_gs.append(A)
        printProgressBar(i, len(trainDir)-1, prefix = 'train gs')
        
    for i in range(len(benignDir)):
        A = np.array(getArray(benignDir[i]))
        G = nx.from_numpy_matrix(A)
        ben_gs.append(G)
        ben_np_gs.append(A)
        printProgressBar(i, len(benignDir)-1, prefix = 'benign gs')
    if npmode: 
        return [mal_np_gs, ben_np_gs, train_np_gs]
    else: 
        return [mal_gs, ben_gs, train_gs]



In [6]:
def findCentrality(graphs):
    ben_gs = graphs[0]
    mal_gs = graphs[1]
    train_gs = graphs[2]
    ben_bs = []
    mal_bs = []
    train_bs = []
    ben_centr = []
    mal_centr = []
    train_centr = []
    for i in ben_gs:
        ben_bs.append(nx.betweenness_centrality(i))
        printProgressBar(ben_gs.index(i),len(ben_gs)-1,prefix = 'benign')
    print('\n')
    for j in mal_gs:
        mal_bs.append(nx.betweenness_centrality(j))
        printProgressBar(mal_gs.index(j),len(mal_gs)-1,prefix = 'malware',)
    print('\n')
    for k in train_gs:
        train_bs.append(nx.betweenness_centrality(k))
        printProgressBar(train_gs.index(k),len(train_gs)-1,prefix = 'train',)
    print('\n')
    for i in range(len(ben_bs)):
        ben_centr.append(list(ben_bs[i].values()))
    for i in range(len(mal_bs)):
        mal_centr.append(list(mal_bs[i].values()))
    for i in range(len(train_bs)):
        train_centr.append(list(train_bs[i].values()))
    return [ben_centr, mal_centr, train_centr]


In [7]:
def calculateDifference(listA, listB, dirs,isBenign):
    csm_val = []
    rootDir = dirs[0]
    benDir = dirs[1]
    trainDir = dirs[2]
    for i in range(len(listA)):
        printProgressBar(i,len(listA)-1,length = min(len(listA), 50),prefix='diff calculation:')
        val = []
        for j in range(len(listB)):
            val.append(1-distance.cosine(listA[i], listB[j]))
        if isBenign:
            csm_val.append([benDir[i].split('\\')[-3],max(val),trainDir[val.index(max(val))].split('\\')[-3]])
        else :
            csm_val.append([rootDir[i].split('\\')[-3],max(val),trainDir[val.index(max(val))].split('\\')[-3]])
    return csm_val

In [8]:
def sortTests(test):
    sortedTests =[]
    for i in range(len(test)):
        srTest = sorted(test[i], key = lambda kv:kv[1],reverse = True)
        sortedTests.append(srTest)
    return sortedTests

In [9]:
def saveFigures(mal_th, ben_th, start, iteration):
    lists = [mal_th,ben_th]
    diffs = []
    v = 0
    for i in range(len(mal_th)):
        v = max(v,abs(mal_th[i]-ben_th[i]))
        diffs.append(abs(mal_th[i]-ben_th[i]))
    print(v)
    d = max(diffs)
    print(d, mal_th[diffs.index(d)])

    for i in lists:
        plt.plot(i,marker= 11)
    plt.title("max dif at %f"%(diffs[diffs.index(d)]))
    plt.legend(["True positives","False positives"])
    plt.yticks(np.arange(0, 1, 0.05))
    plt.xticks(np.arange(0,step, 1))
    plt.ylabel("")
    plt.draw()
    try:
        os.mkdir(str(iteration))
    except OSError:
#         print ("Creation of the directory %s failed" % str(iteration))
        pass
    else:
        print ("Successfully created the directory %s" % str(iteration))
    plt.savefig(str(iteration)+"/"+str(start)+".png")
    plt.clf()



In [10]:
def similarityMetric(sr):
    counterA = 0
    counterB = 0
    counterC = 0
    for i in sr :
        A,B = i[0].split(',')[0], i[0].split(',')[1]
        C,D = i[2].split(',')[0], i[2].split(',')[1]
        if A==C and B==D:
            counterA +=1
        if A==C or B==D:
            counterB +=1
        if A==C or A==D or B == C or B == D:
            counterC += 1
    print("cA = %d (%f) cB= %d (%f) cC =%d (%f)"%(counterA, counterA/len(sr), counterB, counterB/len(sr), counterC, counterC/len(sr)))
    return [counterA/len(sr),counterB/len(sr),counterC/len(sr)]

In [11]:
def detection(ben, mal, itr, step):
    dif = ben[0]-ben[-1]
    thresh = [ben[-1]+i*dif/step for i in range(step)]
    m = np.array(sum(i > thresh for i in mal)/len(mal))
    p = np.array(sum(i > thresh for i in ben)/len(ben))
    for i in [m,p]:
        plt.plot(i)
    plt.yticks(np.arange(0,1.05,0.05))
    plt.draw()
    plt.title('detection at '+str(itr)+ ' iteration')
    plt.savefig(str(itr)+'--btc.png')
    plt.clf()


In [12]:
iterationDirs = [createDirs(i) for i in range(1,6)]

DATASET F1/fold_1/20%_test
DATASET F1/fold_1/80%_train
DATASET F1/specimen_benign/20%_test
DATASET F1\fold_1\20%_test DATASET F1\fold_1\80%_train DATASET F1\specimen_benign\20%_test
DATASET F1/fold_2/20%_test
DATASET F1/fold_2/80%_train
DATASET F1/specimen_benign/20%_test
DATASET F1\fold_2\20%_test DATASET F1\fold_2\80%_train DATASET F1\specimen_benign\20%_test
DATASET F1/fold_3/20%_test
DATASET F1/fold_3/80%_train
DATASET F1/specimen_benign/20%_test
DATASET F1\fold_3\20%_test DATASET F1\fold_3\80%_train DATASET F1\specimen_benign\20%_test
DATASET F1/fold_4/20%_test
DATASET F1/fold_4/80%_train
DATASET F1/specimen_benign/20%_test
DATASET F1\fold_4\20%_test DATASET F1\fold_4\80%_train DATASET F1\specimen_benign\20%_test
DATASET F1/fold_5/20%_test
DATASET F1/fold_5/80%_train
DATASET F1/specimen_benign/20%_test
DATASET F1\fold_5\20%_test DATASET F1\fold_5\80%_train DATASET F1\specimen_benign\20%_test


In [13]:
graphs = [createGraphs(iterationDirs[i],False)for i in range(len(iterationDirs))]

malware gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


train gs |████████████████████████████------------------------------------------------------------------------| 28.0% 

train gs |███████████████████████████████████████████████████████---------------------------------------------| 55.6% 

train gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
benign gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


malware gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


...

train gs |████████████████████████████████████████████████████████████████████████████████████████------------| 88.5% 

train gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
benign gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


malware gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


train gs |████████████████████████████████████████████--------------------------------------------------------| 44.0% 

train gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
benign gs |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


In [14]:
graphCentr = [findCentrality(graphs[i]) for i in range(len(graphs))]

benign |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


malware |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


train |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


benign |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


malware |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


train |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


benign |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


malware |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


train |████████████████████████████████

In [15]:
t = time.time()
malwareTests = [calculateDifference(graphCentr[i][0],graphCentr[i][2], 
                                    iterationDirs[i], False) for i in range(len(graphCentr))]
print("malware took %f seconds"%(time.time()-t))

diff calculation: |██████████████████████████████████████████████████| 100.0% 
diff calculation: |██████████████████████████████████████████████████| 100.0% 
diff calculation: |██████████████████████████████████████████████████| 100.0% 
diff calculation: |██████████████████████████████████████████████████| 100.0% 
diff calculation: |██████████████████████████████████████████████████| 100.0% 
malware took 133.497728 seconds


In [16]:
benignTests = [calculateDifference(graphCentr[i][1],graphCentr[i][2], 
                                   iterationDirs[i], True) for i in range(len(graphCentr))]

diff calculation: |███████████████████████████████████| 100.0% 
diff calculation: |███████████████████████████████████| 100.0% 
diff calculation: |███████████████████████████████████| 100.0% 
diff calculation: |███████████████████████████████████| 100.0% 
diff calculation: |███████████████████████████████████| 100.0% 


In [17]:
sortBenignTests= sortTests(benignTests)

In [18]:
sortMalwareTests =  sortTests(malwareTests)

In [19]:
malwares =  [[sortMalwareTests[j][i][1] for i in range(len(sortMalwareTests[j]))] 
             for j in range(len(sortMalwareTests))]
benigns =  [[sortBenignTests[j][i][1] for i in range(len(sortBenignTests[j]))] 
             for j in range(len(sortBenignTests))]

In [20]:
step = 20
for i in range(len(benigns)):
    detection(benigns[i], malwares[i], i, step)

<Figure size 432x288 with 0 Axes>

In [21]:
similarities = [similarityMetric(sortMalwareTests[i]) for i in range(len(sortMalwareTests))]
for i in similarities :
    print(i)
print('-------------------------------\n')
counts = [[i[j] for i in similarities] for j in range(len(i))]
avg = [statistics.mean(i) for i in counts]

for j in avg : print(j)

cA = 216 (0.414587) cB= 297 (0.570058) cC =314 (0.602687)
cA = 217 (0.416507) cB= 318 (0.610365) cC =328 (0.629559)
cA = 252 (0.483685) cB= 321 (0.616123) cC =339 (0.650672)
cA = 217 (0.416507) cB= 292 (0.560461) cC =309 (0.593090)
cA = 191 (0.361059) cB= 262 (0.495274) cC =286 (0.540643)
[0.4145873320537428, 0.5700575815738963, 0.6026871401151631]
[0.4165067178502879, 0.6103646833013435, 0.6295585412667947]
[0.4836852207293666, 0.6161228406909789, 0.6506717850287908]
[0.4165067178502879, 0.5604606525911708, 0.5930902111324377]
[0.3610586011342155, 0.4952741020793951, 0.5406427221172023]
-------------------------------

0.41846891792358015
0.570455972047357
0.6033300799320777


In [22]:
print(counts)

[[0.4145873320537428, 0.4165067178502879, 0.4836852207293666, 0.4165067178502879, 0.3610586011342155], [0.5700575815738963, 0.6103646833013435, 0.6161228406909789, 0.5604606525911708, 0.4952741020793951], [0.6026871401151631, 0.6295585412667947, 0.6506717850287908, 0.5930902111324377, 0.5406427221172023]]
